In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras import backend as K
print(K.backend())

tensorflow


In [ ]:
# Function to visualize original and augmented images
def visualize_augmentations(preprocessing_pipeline, x_train, num_images=5):
    plt.figure(figsize=(10, 5))
    
    for i in range(num_images):
        # Original image
        original_image = x_train[i]
        
        # Apply preprocessing pipeline to the image
        augmented_image = preprocessing_pipeline(tf.expand_dims(original_image, axis=0))
        
        # Plot original image
        plt.subplot(2, num_images, i + 1)
        plt.imshow(original_image)
        plt.title("Original")
        plt.axis("off")
        
        # Plot augmented image
        plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(tf.squeeze(augmented_image).numpy())
        plt.title("Augmented")
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

In [22]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

preprocessing_pipeline = keras.Sequential([
    layers.RandomZoom(0.2),
    layers.RandomRotation(0.2),
    layers.RandomFlip(),
])

inputs = keras.Input((32, 32, 3))

x = preprocessing_pipeline(inputs)

x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Dropout(0.2)(x)

x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Flatten()(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='cifar10_model')
model.summary()
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    metrics=['accuracy']
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=20,
    min_delta=0.001,
    restore_best_weights=True
    )

visualize_augmentations(preprocessing_pipeline, x_train, num_images=5)

history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_split=0.2, 
                    callbacks=[early_stopping])

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Model: "cifar10_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 30, 30, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 30, 30, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 15, 15, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 15, 15, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 13, 13, 64)     │         9,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10)             │        23,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,098 (129.29 KB)

 Trainable params: 32,938 (128.66 KB)

 Non-trainable params: 160 (640.00 B)

Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.2478 - loss: 2.2976 - val_accuracy: 0.2615 - val_loss: 1.9642
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.3374 - loss: 1.8560 - val_accuracy: 0.3679 - val_loss: 1.7219
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.3686 - loss: 1.7589 - val_accuracy: 0.4097 - val_loss: 1.6724
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.3881 - loss: 1.7053 - val_accuracy: 0.3997 - val_loss: 1.7207
Epoch 5/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.3943 - loss: 1.6810 - val_accuracy: 0.4239 - val_loss: 1.6609
Epoch 6/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.4087 - loss: 1.6476 - val_accuracy: 0.4087 - val_loss: 1.7159
Epoch 7/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.4140 - loss: 1.6229 - val_accuracy: 0.4204 - val_loss: 1.6708
Epoch 8/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.4236 - loss: 1.6086 - val_accu

In [ ]:
# Confusion matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


ValueError: Found input variables with inconsistent numbers of samples: [100000, 10000]